Load the Weather-Routing-Tool

In [ ]:
import WeatherRoutingTool

Set enviroment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv() 

Run the Weather-Routing-Tool with the given template 

In [ ]:
%run cli.py -f config.template.json

Display the route

In [ ]:
import os
import json
from ipyleaflet import Map, GeoJSON

m = Map(center=[39.926688, 7.930542], zoom=5.5)

with open("min_time_route.json") as f:
    data = json.load(f)

# Add the GeoJSON layer
geo_json = GeoJSON(data=data)

# Add the GeoJSON layer to the map
m.add_layer(geo_json)
m

You can go through the routing steps by moving the slider

In [ ]:
import os
import re
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets

# Resize images
%run resize_Images_WRT.py

# Define key for sorted function
def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


# Set path to Images folder
image_folder = "Images-WRT"
image_files = sorted([
    f for f in os.listdir(image_folder)
    if f.lower().endswith('.png')
    ], key=natural_key)

# Output-Widget for the image 
image_output = widgets.Output()

# Function for displaying the images
def show_image(index):
    image_path = os.path.join(image_folder, image_files[index])
    with image_output:
        image_output.clear_output(wait=True)
        img = Image.open(image_path)
        display(img)

# Defining the slider
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(image_files) - 1,
    step=1,
    orientation='vertical',
    description='routing',
    continuous_update=True,
    layout=widgets.Layout(height='300px')
)

# Callback for the slider
slider.observe(lambda change: show_image(change['new']), names='value')

# Layout
ui = widgets.HBox([slider,image_output], layout=widgets.Layout(align_items='center'))

# Show first image
show_image(0)

# Show ui
display(ui)

TraitError: setting min > max

You can start youre own routing by setting the two map markers (startpoint:green, endpoint:red) and pressing 'start routing'. If want to set a new route press 'new route'.

In [ ]:
import json
import os
from ipyleaflet import Map, Marker, AwesomeIcon, GeoJSON
import ipywidgets as widgets
from ipywidgets import Button

# Define the look of the map markers
icon1 = AwesomeIcon(
    name='map-pin',       
    marker_color='green',       
    icon_color='white',       
    spin=False
)

icon2 = AwesomeIcon(
    name='map-pin',       
    marker_color='red',       
    icon_color='white',       
    spin=False
)

# Define the map and the map markers
center = ([39.926688, 7.930542])
m = Map(center=center, zoom=6)
marker1 = Marker(location=([39.926688, 5]), draggable=True, icon=icon1)
marker2 = Marker(location=([39.926688, 10.5]), draggable=True, icon=icon2)

m.add(marker1)
m.add(marker2)

# Define Buttons to start the routing and to set a new route
button1= widgets.Button(description="start routing")
button2= widgets.Button(description="new route")

# Function that deletes the last route and sets teh map markers to their default position
def on_button2_clicked(b):
    marker1.location=(39.926688, 5)
    marker2.location=(39.926688, 10.5)
    m.remove(geo_json)
    m.add(marker1)
    m.add(marker2)

# Function thet rewrites the config file so the WRT calculates the route between the given map markers,
# starts the WRT and displays the calculated route
def on_button1_clicked(b):
    a,b=marker1.location
    c,d=marker2.location
    with open('config.template.json', 'r+') as f:
        data = json.load(f)
        data['DEFAULT_ROUTE'] = [a,b,c,d]
    with open('config.template.json', 'w') as file: 
        json.dump(data, file, indent=4)
    %run cli.py -f config.template.json
    with open("min_time_route.json") as f:
        data = json.load(f)
    global geo_json
    geo_json = GeoJSON(data=data)
    m.remove(marker1)
    m.remove(marker2)
    m.add(geo_json)

# Add callbacks for the buttons and display them and he map 
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)
display(button1)
display(button2)
m